# Juntar variantes de Juan con las de ClinVar

In [1]:
import pandas as pd
import re


### Cargar archivos de variantes 

### ClinVar

In [3]:

clinvar_df = pd.read_csv("clinvar_result.txt", sep="\t", engine="python", dtype=str)


# Ver un pantallazo
print("CLINVAR:")
print(clinvar_df.head())


CLINVAR:
                                        Name Gene(s) Protein change  \
0    NM_000453.3(SLC5A5):c.17C>T (p.Thr6Ile)  SLC5A5            T6I   
1    NM_000453.3(SLC5A5):c.19G>A (p.Gly7Arg)  SLC5A5            G7R   
2   NM_000453.3(SLC5A5):c.52G>A (p.Gly18Arg)  SLC5A5           G18R   
3   NM_000453.3(SLC5A5):c.67A>T (p.Met23Leu)  SLC5A5           M23L   
4  NM_000453.3(SLC5A5):c.103G>A (p.Gly35Arg)  SLC5A5           G35R   

                  Condition(s)     Accession GRCh37Chromosome GRCh37Location  \
0  Thyroid dyshormonogenesis 1  VCV000891359               19       17983145   
1  Thyroid dyshormonogenesis 1  VCV000891360               19       17983147   
2                not specified  VCV003339135               19       17983180   
3      Inborn genetic diseases  VCV003165258               19       17983195   
4      Inborn genetic diseases  VCV002334784               19       17983231   

  GRCh38Chromosome GRCh38Location VariationID  ... Germline classification  \
0    

### Las que tenia juan

In [5]:
# Cargar variantes estructurales
juan_df = pd.read_csv("JuanVariants.csv")

# Verificamos las primeras filas
print(juan_df.head())
print("\nColumnas disponibles:", juan_df.columns.tolist())

  Cambio de aminoácido      Fuente Localización     Actividad
0            p.Arg9Ala  Artificial     Membrana    Activa (1)
1           p.Gly18Arg    Paciente     Membrana      Inactiva
2           p.Ser43Ala  Artificial     Membrana  Activa (0.6)
3           p.Ser43Asp  Artificial     Membrana    Activa (1)
4           p.Ser43Thr  Artificial     Membrana    Activa (1)

Columnas disponibles: ['Cambio de aminoácido', 'Fuente', 'Localización', 'Actividad']


In [3]:
# === 1. Cargar los datos ===
juan_df = pd.read_csv("JuanVariants.csv")
clinvar_df = pd.read_csv("clinvar_result.txt", sep="\t", dtype=str)  # Ajustá sep si no es tabulado

# === 2. Extraer la variante p.XxxYyy de ClinVar ===
def extraer_protein_change(texto):
    if pd.isna(texto):
        return None
    match = re.search(r"\(p\.[A-Za-z]{3}\d+[A-Za-z]{3}\)", texto)
    if match:
        return match.group(0)[1:-1]  # Quita los paréntesis
    return None

# Aplica la función
clinvar_df["Cambio de aminoácido"] = clinvar_df.iloc[:, 0].apply(extraer_protein_change)  # usa la 1ra col

# Eliminar NaN y duplicados
clinvar_variants = clinvar_df["Cambio de aminoácido"].dropna().unique()
juan_variants = juan_df["Cambio de aminoácido"].dropna().unique()

# === 3. Reporte de cantidades ===
print(f"Variantes en Juan: {len(juan_variants)}")
print(f"Variantes en ClinVar (p.XxxYyy extraídas): {len(clinvar_variants)}")


Variantes en Juan: 263
Variantes en ClinVar (p.XxxYyy extraídas): 76


In [4]:
# === 4. Verificar coincidencias ===
# Cuáles de ClinVar ya están en Juan
en_juan = [v for v in clinvar_variants if v in juan_variants]
no_en_juan = [v for v in clinvar_variants if v not in juan_variants]

print(f"Variantes de ClinVar también en Juan: {len(en_juan)}")
print(f"Variantes de ClinVar NO presentes en Juan: {len(no_en_juan)}")

Variantes de ClinVar también en Juan: 16
Variantes de ClinVar NO presentes en Juan: 60


In [7]:
# Filtrar las que no están en Juan
clinvar_unicas_df = clinvar_df[clinvar_df["Cambio de aminoácido"].isin(no_en_juan)].copy()

# === 3. Normalizar columnas antes del merge ===

# Añadir 'Fuente' a ambos
juan_df["Fuente"] = "Juan"
clinvar_unicas_df["Fuente"] = "ClinVar"

# Unir columnas
todas_las_columnas = sorted(set(juan_df.columns).union(set(clinvar_unicas_df.columns)))

# Asegurar que ambos tengan todas las columnas
juan_df = juan_df.reindex(columns=todas_las_columnas)
clinvar_unicas_df = clinvar_unicas_df.reindex(columns=todas_las_columnas)

# === 4. Concatenar ===
df_combinado = pd.concat([juan_df, clinvar_unicas_df], ignore_index=True)

# === 5. Resultado ===
print(f"Total de variantes combinadas: {df_combinado.shape[0]}")
print(df_combinado.head())

# Opcional: guardar
df_combinado.to_csv("VariantesCombinadas.csv", index=False)

Total de variantes combinadas: 323
  Accession     Actividad AlleleID(s) Cambio de aminoácido Canonical SPDI  \
0       NaN    Activa (1)         NaN            p.Arg9Ala            NaN   
1       NaN      Inactiva         NaN           p.Gly18Arg            NaN   
2       NaN  Activa (0.6)         NaN           p.Ser43Ala            NaN   
3       NaN    Activa (1)         NaN           p.Ser43Asp            NaN   
4       NaN    Activa (1)         NaN           p.Ser43Thr            NaN   

  Condition(s) Fuente GRCh37Chromosome GRCh37Location GRCh38Chromosome  ...  \
0          NaN   Juan              NaN            NaN              NaN  ...   
1          NaN   Juan              NaN            NaN              NaN  ...   
2          NaN   Juan              NaN            NaN              NaN  ...   
3          NaN   Juan              NaN            NaN              NaN  ...   
4          NaN   Juan              NaN            NaN              NaN  ...   

  Oncogenicity date last ev

In [8]:
# Reordenar columnas: que "Cambio de aminoácido" esté primera
columnas_ordenadas = ["Cambio de aminoácido"] + [col for col in df_combinado.columns if col != "Cambio de aminoácido"]
df_combinado = df_combinado[columnas_ordenadas]

In [9]:
df_combinado.to_csv("VariantesCombinadas.csv", index=False)
print("Archivo guardado como VariantesCombinadas.csv ✅")

Archivo guardado como VariantesCombinadas.csv ✅


In [10]:
# Buscar duplicados en "Cambio de aminoácido"
duplicados = df_combinado[df_combinado.duplicated(subset=["Cambio de aminoácido"], keep=False)]

if duplicados.empty:
    print("✅ No hay variantes repetidas en 'Cambio de aminoácido'.")
else:
    print(f"⚠️ Hay {duplicados.shape[0]} filas con variantes repetidas:")
    print(duplicados.sort_values("Cambio de aminoácido"))

✅ No hay variantes repetidas en 'Cambio de aminoácido'.
